In [15]:
from flask import Flask, request
from flask_cors import CORS, cross_origin
from flask_restful import Resource, Api
from json import dumps
from flask_jsonpify import jsonify
import json
import pymongo
import urllib
from collections import Counter
import requests
from flask_swagger_ui import get_swaggerui_blueprint

In [16]:
surveyTemplate = {
        'userName' : "Hemanth",
        'surveyName' : "test",
        'surveyForm':{"0":{
            "question":"qsn1",
            "questionType":"radio",
            "options":"sd,sdf,sv",
            "rangeFrom" :"1",
            "rangeFrom" :"20"
        },
                "1":{
            "qsn":"qsn2",
            "type":"texxt",
            "options":["a","c"]
        }
            
        }
    }

surveyEntry= {
        'userName' : "Hemanth",
        'surveyName' : "test",
        'entryForm':{"0":"ans1",
        "1":"ans1"   
        }
    }

In [17]:
def checkSurveyExists(surveyName):
    db=client.survey
    form=db.SurveyForms.find_one({'surveyName': surveyName})
    if(form):
        return True
    return False


def mergeSingleSurveyEntry(surveyTemplate,surveyEntry):
    form=surveyTemplate["surveyForm"]
    for key in form:
        form[key]["answer"]=surveyEntry["entryForm"][key]
    return form


def mergeSurveyEntry(surveyTemplate,surveyEntry):
    form=surveyTemplate
    for key in form["surveyForm"]:
        form["surveyForm"][key]["answers"]=[]
        
    form["users"]=[]
    for entry in surveyEntry:
        form["users"].append(entry["userName"])
        for key in form["surveyForm"]:
            form["surveyForm"][key]["answers"].append(entry["entryForm"][key])
            
    for key in form["surveyForm"]:
        form["surveyForm"][key]["answers"]=Counter(form["surveyForm"][key]["answers"])
        
    return form



def verifyAccessToken(access_token):
    URL="http://127.0.0.1:5001/authorize"
    response=requests.post(url = URL,headers={'Authorization': "access_token"})
    if(response.status_code==200):
        return True
    else:
        return False

In [ ]:
app = Flask(__name__)
api = Api(app)

CORS(app)
client = pymongo.MongoClient("mongodb+srv://hemanth:"+ urllib.quote("Hemanth@123")+"@cluster0-59obx.mongodb.net/test?retryWrites=true&w=majority")



### swagger specific ###
SWAGGER_URL = '/swagger'
API_URL = '/static/swagger.json'
SWAGGERUI_BLUEPRINT = get_swaggerui_blueprint(
    SWAGGER_URL,
    API_URL,
    config={
        'app_name': "Seans-Python-Flask-REST-Boilerplate"
    }
)
app.register_blueprint(SWAGGERUI_BLUEPRINT, url_prefix=SWAGGER_URL)  


@app.route("/")
def hello():
    return jsonify({'text':'Hello World!'})


@app.route("/createSurvey",methods=['POST'])
def createSurvey(): 
    
    verifyAccessToken(request.headers["Authorization"])
    if not(verifyAccessToken):
        return json.dumps({
              "error": "access_denied"
            }), 401
    
    db=client.survey
    params=request.json
    if(checkSurveyExists(params["surveyName"])):
        return jsonify({'response':'SurveyName already Exists'}),400
    
    surveyDetails={}
    surveyDetails["userName"]=params["userName"]
    surveyDetails["surveyName"]=params["surveyName"]
    surveyDetails["surveyDescription"]=params["surveyDescription"]
    surveyDetails["surveyForm"]=params["surveyForm"]

    
    result=db.SurveyForms.insert_one(surveyDetails)
    return  jsonify({'response':'Survey successfully created'})


@app.route("/getSurvey",methods=['POST'])
def SurveyForm():    
    db=client.survey
    params=request.json

    if not(checkSurveyExists(params["surveyName"])):
        return jsonify({'response':'SurveyName doesnt Exists'})
    
    result=db.SurveyForms.find_one({'surveyName': params["surveyName"]})
    del result['_id']
    return jsonify(result)


@app.route("/fillSurvey",methods=['POST'])
def SurveyEntry():
    db=client.survey
    params=request.json
    if not(checkSurveyExists(params["surveyName"])):
        return jsonify({'response':'SurveyName doesnt Exist'})
    
    surveyDetails={}
    surveyDetails["userName"]=params["userName"]
    surveyDetails["surveyName"]=params["surveyName"]
    surveyDetails["entryForm"]=params["entryForm"]
    
    if(db.SurveyEntries.find_one({'surveyName': params["surveyName"],'userName':params["userName"]})):
        return  jsonify({'response':'User has already filled the survey'}),400
    
    db.SurveyEntries.insert_one(surveyDetails)
    return  jsonify({'response':'Survey successfully filled'})


@app.route("/getAllSurveys",methods=['GET'])
def getAllServeys():
    db=client.survey
    surveys=[]
    for i in db.SurveyForms.find():
        del i['_id']
        surveys.append(i)
   
    return jsonify({"results":surveys})

@app.route("/getUserSurveys",methods=['POST'])
def getUserServeys():
    params=request.json
    db=client.survey
    surveys=[]
    for i in db.SurveyForms.find({'userName': params["userName"]}):
        del i['_id']
        surveys.append(i)
   
    return jsonify({"results":surveys})

@app.route("/getSurveyEntries",methods=['POST'])
def getSurveyEntries():
    db=client.survey
    params=request.json
    print(params)
    if not(checkSurveyExists(params["surveyName"])):
        return jsonify({'response':'SurveyName doesnt Exists'})
       
    surveyEntries=[]
    print(db.SurveyEntries.find({'surveyName': params["surveyName"]}))
    for i in db.SurveyEntries.find({'surveyName': params["surveyName"]}):
        del i['_id']
        surveyEntries.append(i)
        
    print(surveyEntries)
    return jsonify({"results":surveyEntries})

# @app.route("/surveystats")
# def SurveyStatistics():
#     db=client.survey
#     data=json.loads(str(request.data))
#     if(!checkSurveyExists(request.surveyName)):
#         return jsonify({'response':'SurveyName doesnt Exists'})
    
#     surveyEntries=db.SurveyEntries.find({'surveyName': request.surveyName})
#     surveyTemplate=db.SurveyForms.find({'surveyName': request.surveyName})

#     return surveyEntries

@app.route("/surveystat")
def getByUserSurvey():
    db=client.SurveyEntries
    params=request.json
    print(params)
    if not(checkSurveyExists(params["surveyName"])):
        return jsonify({'response':'SurveyName doesnt Exists'})
    
    surveyEntry=db.SurveyEntries.find({'surveyName': params["surveyName"],'userName':params["userName"]})
    surveyTemplate=db.SurveyForms.find({'surveyName': params["surveyName"]})
    
    return mergeSurventry(surveyTemplate,surveyEntry)

if __name__ == '__main__':
   app.run(port=5002)



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5002/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Oct/2019 01:22:14] "OPTIONS /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 01:22:17] "POST /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 01:22:21] "OPTIONS /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 01:22:22] "POST /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 01:22:28] "OPTIONS /fillSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 01:22:28] "POST /fillSurvey HTTP/1.1" 400 -
127.0.0.1 - - [19/Oct/2019 01:22:32] "POST /fillSurvey HTTP/1.1" 400 -
127.0.0.1 - - [19/Oct/2019 01:22:35] "OPTIONS /fillSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 01:22:35] "POST /fillSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 01:23:21] "OPTIONS /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 01:23:21] "OPTIONS /getAllSurveys HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 01:23:21] "POST /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 01:23:23] "GET /getAllSurveys HTTP/1.1" 200 -


{u'surveyName': u'Testing Survey'}


127.0.0.1 - - [19/Oct/2019 01:33:04] "POST /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 01:33:04] "POST /getSurveyEntries HTTP/1.1" 200 -


[{u'userName': u'test', u'surveyName': u'Testing Survey', u'entryForm': {u'1': u'Radio2', u'0': u'test', u'3': u'40.31', u'2': u'DropDown3'}}, {u'userName': u'hemanth', u'surveyName': u'Testing Survey', u'entryForm': {u'1': u'Radio2', u'0': u'test', u'3': u'42.73', u'2': u'DropDown2'}}, {u'userName': u'user', u'surveyName': u'Testing Survey', u'entryForm': {u'1': u'Radio1', u'0': u'testing the results', u'2': u'DropDown3'}}, {u'userName': u'user2', u'surveyName': u'Testing Survey', u'entryForm': {u'1': u'Radio3', u'0': u'233333', u'3': u'42.95', u'2': u'DropDown3'}}, {u'userName': u'Final Test', u'surveyName': u'Testing Survey', u'entryForm': {u'1': u'Radio3', u'0': u'final test', u'3': u'93.61', u'2': u'DropDown3'}}, {u'userName': u'nanda', u'surveyName': u'Testing Survey', u'entryForm': {u'1': u'Radio1', u'0': u'nanda', u'3': u'69.60', u'2': u'DropDown3'}}, {u'userName': u'nanda', u'surveyName': u'Testing Survey', u'entryForm': {u'1': u'Radio1', u'0': u'nanda', u'3': u'69.60', u'2': 

In [ ]:
surveyTemplate = {
        'userName' : "Hemanth",
        'surveyName' : "test",
        'surveyForm':{"0":{
            "qsn":"qsn1",
            "type":"radio",
            "options":["a","b"]
        },
                "1":{
            "qsn":"qsn2",
            "type":"texxt",
            "options":["a","c"]
        }
            
        }
    }

surveyEntry= [{
        'userName' : "Hemanth",
        'surveyName' : "test",
        'entryForm':{"0":"ans1",
        "1":"ans1" }
        },{
        'userName' : "Mounika",
        'surveyName' : "test",
        'entryForm':{"0":"ans1",
        "1":"ans12" }
        }]

